In [14]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import numpy as np

In [15]:
# Carregando o conjunto de dados do Titanic
data = pd.read_csv('train.csv')

In [16]:
# Extraindo as letras do conves e transformando em números
data['Cabin'] = data['Cabin'].str.extract(r'([a-gA-G]+)')

# Preenchendo idede ausente
data.loc[data.Age.isnull(), 'Age'] = data.Age.mean()


# Convertendo categorio em numerico
data['Sex'] = data.Sex.apply(lambda x: 0 if x == 'male' else 1).astype(int)

In [17]:
# Dividindo os dados em um conjunto rotulado e outro não rotulado
labeled_data = data[data['Cabin'].notnull()]
unlabeled_data = data[data['Cabin'].isnull()]

In [18]:
# Selecionando as colunas relevantes para treinamento
feature_columns = ['Pclass', 'Sex', 'Age', 'Fare']
X_labeled = labeled_data[feature_columns]
y_labeled = labeled_data['Cabin']
X_unlabeled = unlabeled_data[feature_columns]

In [19]:
# Dividindo o conjunto rotulado em treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(X_labeled, y_labeled, test_size=0.2, random_state=42)

# Criando dois modelos de árvore de decisão com diferentes conjuntos de características
model_1 = DecisionTreeClassifier(random_state=42)
model_2 = DecisionTreeClassifier(random_state=42)

In [20]:
# Treinando os modelos nos dados rotulados
model_1.fit(X_train, y_train)
model_2.fit(X_train[['Pclass', 'Sex']], y_train)  # Usamos duas características específicas como exemplo

# Classificando os dados não rotulados e selecionando os mais confiantes
unlabeled_predictions_1 = model_1.predict(X_unlabeled)
unlabeled_predictions_2 = model_2.predict(X_unlabeled[['Pclass', 'Sex']])

In [21]:
confident_indices_1 = np.where(unlabeled_predictions_1 == unlabeled_predictions_2)[0]
confident_unlabeled_data = X_unlabeled.iloc[confident_indices_1]

In [22]:
# Adicionando os exemplos mais confiantes ao conjunto rotulado e treinando novamente os modelos
X_train = pd.concat([X_train, confident_unlabeled_data])
y_train = pd.concat([y_train, pd.Series(unlabeled_predictions_1[confident_indices_1])])

model_1.fit(X_train, y_train)
model_2.fit(X_train[['Pclass', 'Sex']], y_train)  # Usamos as mesmas características selecionadas anteriormente

DecisionTreeClassifier(random_state=42)

In [23]:
# Agora podemos usar os modelos treinados para fazer previsões em novos dados
teste_filtrado = pd.read_csv('teste_filtrado.csv')
treino_filtrado = pd.read_csv('treino_filtrado.csv')

treino_filtrado['Sex'] = treino_filtrado.Sex.apply(lambda x: 0 if x == 'male' else 1).astype(int)
teste_filtrado['Sex'] = teste_filtrado.Sex.apply(lambda x: 0 if x == 'male' else 1).astype(int)

treino_predictions_1 = model_1.predict(treino_filtrado[feature_columns])
treino_predictions_2 = model_2.predict(treino_filtrado[['Pclass', 'Sex']])

teste_predictions_1 = model_1.predict(teste_filtrado[feature_columns])
teste_predictions_2 = model_2.predict(teste_filtrado[['Pclass', 'Sex']])


# print("Modelo 1 previu as classes:", predictions_1)
# print("Modelo 2 previu as classes:", predictions_2)


In [24]:
teste_filtrado['Cabin'] = teste_predictions_1
treino_filtrado['Cabin'] = treino_predictions_1

In [25]:
teste_filtrado.to_csv('teste_filtrado.csv', index=False)
treino_filtrado.to_csv('treino_filtrado.csv', index=False)